# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [1]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [2]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [3]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [4]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [5]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [6]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.08166817e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [27]:
# ... ENTER YOUR CODE HERE ...

def Jacobi_iteration(A, b, n_iter = 50):
    
    diag_1d = np.diag(A)
    B = -A.copy()
    np.fill_diagonal(B, 0)
    invD = np.diag(1./diag_1d)
    BB = invD @ B 
    c = invD @ b
    
    x = np.ones(n)
    for _ in range(n_iter):
        x = BB @ x + c
    return x

In [28]:
import numpy as np
rndm = np.random.RandomState(22)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [29]:
x = Jacobi_iteration(A, b)
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@x, b)
np.testing.assert_allclose(x, xx)

In [30]:
xx

array([-0.00275305,  0.03518762,  0.01474805,  0.00462925,  0.04705064,
        0.02730976,  0.03229428,  0.00613677,  0.04186681,  0.00770128])

In [31]:
x

array([-0.00275305,  0.03518762,  0.01474805,  0.00462925,  0.04705064,
        0.02730976,  0.03229428,  0.00613677,  0.04186681,  0.00770128])

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [37]:
# ... ENTER YOUR CODE HERE ...

def seidel_iteration(A, b, n_iter = 50):
    size = b.shape[0]
    x = np.ones(size)
    
    for _ in range(n_iter):
        for k in range(size):
              x[k] = (b[k]-np.dot(A[k][:k], x[:k])- np.dot(A[k][k+1:], x[k+1:]))/A[k,k]
    return x

In [38]:
import numpy as np
rndm = np.random.RandomState(22)
x = seidel_iteration(A, b)
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@x, b)
np.testing.assert_allclose(x, xx)
n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [39]:
x = seidel_iteration(A, b)
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@x, b)
np.testing.assert_allclose(x, xx)

In [40]:
xx

array([ 0.0375434 ,  0.04773325, -0.00346755,  0.03540281,  0.05802232,
        0.03777484,  0.01441998,  0.00841281,  0.01143376,  0.01180073])

In [41]:
x

array([ 0.0375434 ,  0.04773325, -0.00346755,  0.03540281,  0.05802232,
        0.03777484,  0.01441998,  0.00841281,  0.01143376,  0.01180073])

# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [48]:
# ... ENTER YOUR CODE HERE ...

def minimum_residual_scheme(A, b, n_iter = 50):
    x = np.ones(b.shape[0])
    
    for _ in range(n_iter):
        r = A @ x - b
        k = np.dot(r, A @ r) / np.linalg.norm(A @ r)**2
        x = x - k*r
        
    return x

In [57]:
import numpy as np
rndm = np.random.RandomState(0)

n = 15
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [58]:
x = minimum_residual_scheme(A, b)
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@x, b)
np.testing.assert_allclose(x, xx)

In [59]:
x

array([ 0.05218857,  0.03537676,  0.00509974,  0.05414701,  0.04159786,
        0.01136444,  0.0050098 ,  0.02182408, -0.00902046,  0.00101749,
        0.01680991,  0.01550099,  0.02108168,  0.01027258,  0.03007672])

In [60]:
xx

array([ 0.05218857,  0.03537676,  0.00509974,  0.05414701,  0.04159786,
        0.01136444,  0.0050098 ,  0.02182408, -0.00902046,  0.00101749,
        0.01680991,  0.01550099,  0.02108168,  0.01027258,  0.03007672])